In [3]:
## 读取数据集

import os

os.makedirs(os.path.join('.', 'data'), exist_ok=True)
data_file = os.path.join('.', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

# 如果没有安装pandas，只需取消对以下行的注释来安装pandas： pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [18]:
## 处理缺失值

# NaN代表缺失值
# 常见的处理缺失值的方法包括 插值法 和 删除法。
# 插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值

# 插值法： 用同列的平均值代替缺失值
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]  # 将三列分为前两列，和最后一列，两组
print(inputs)
print(outputs)

# Alley这列是离散值，总共有两个值：Pave和nan，pandas会把这列转为两列，Alley_Pave和Alley_nan
# 所以Pave的列，其Alley_Pave为True(1)，Alley_nan为False(0)。而NaN列，其Alley_Pave为0，Alley_nan为1
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

# 用均值代替缺失值
try:
    inputs = inputs.fillna(inputs.mean())  # inputs.select_dtypes(include="number").mean()
    # inputs['Alley_Pave'] = inputs["Alley_Pave"].astype(int)
    # inputs['Alley_nan'] = inputs["Alley_nan"].astype(int)
    print(inputs)
except Exception as e:
    print(e)

   NumRooms Alley
0       NaN  Pave
1       2.0   NaN
2       4.0   NaN
3       NaN   NaN
0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64
   NumRooms  Alley_Pave  Alley_nan
0       NaN        True      False
1       2.0       False       True
2       4.0       False       True
3       NaN       False       True
   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0       False       True
2       4.0       False       True
3       3.0       False       True


In [19]:
## 转为张量格式

import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
Y = torch.tensor(outputs.to_numpy(dtype=float))
print(X)
print(Y)

tensor([[3., 1., 0.],
        [2., 0., 1.],
        [4., 0., 1.],
        [3., 0., 1.]], dtype=torch.float64)
tensor([127500., 106000., 178100., 140000.], dtype=torch.float64)


In [25]:
## 习题： 删除缺失值最多的列 及 将预处理后的数据集转换为张量格式。
data = pd.read_csv(data_file)
print("data:", data)

# 查找每列中的 NaN 值数量
nan_counts = pd.isna(data).sum()
print("nan_counts:", nan_counts)
# 找出 NaN 值最多的列
max_nan_col = nan_counts.idxmax()
print("max_nan_col:", max_nan_col)
# 删除方法
data.drop(max_nan_col, axis=1, inplace=True)
print("data:", data)

# 将预处理后的数据集转换为张量格式。
inputs, outputs = data.iloc[:, :-1], data.iloc[:, -1]
inputs = inputs.fillna(inputs.mean())
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

# DataFrame对象的values，就是ndarray类型
x, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
print(x)
print()
print(y)

data:    NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000
nan_counts: NumRooms    2
Alley       3
Price       0
dtype: int64
max_nan_col: Alley
data:    NumRooms   Price
0       NaN  127500
1       2.0  106000
2       4.0  178100
3       NaN  140000
   NumRooms
0       3.0
1       2.0
2       4.0
3       3.0
<class 'numpy.ndarray'>
tensor([[3.],
        [2.],
        [4.],
        [3.]], dtype=torch.float64)

tensor([127500, 106000, 178100, 140000])
